In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nose.tools import *

np.random.seed(24680)

Write your imports in the cell below.

In [3]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC

# Ensemble Models and Support Vector Machines Lab
## Training and comparing different algorithms

Once again, we'll work with the bank dataset. This time, the data preprocessing steps have been done for you.

The goal is to try and improve our predictions (if they can be improved at all) using different types of algorithms.

### 1. Read the data (1 point)
This time you only need to read the data. The indicator variables have been separated out for you.

Read the dataset and save it in the variable `bank_data`. The target column is `y`. Use the variables `bank_attributes` and `bank_labels` to save the attributes (explanatory variables, features, predictors), and labels (`y`).

In [4]:
bank_data = pd.read_csv("data/bank.csv")
bank_data.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,y
0,30,1787,19,79,1,-1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,33,4789,11,220,1,339,4,0,0,0,...,0,1,0,0,0,1,0,0,0,0
2,35,1350,16,185,1,330,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,30,1476,3,199,4,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,59,0,5,226,1,-1,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0


In [5]:
bank_attributes = bank_data.drop("y", axis = 1)
bank_labels = bank_data.y

In [6]:
assert_is_not_none(bank_data)
assert_is_not_none(bank_attributes)
assert_is_not_none(bank_labels)

### 2. Normalize the data (1 point)
Because both forests and SVMs are sensitive to non-scaled data, we need to normalize our dataset first.

Rescale all columns in `bank_attributes` so they have mean = 0 and variance = 1. You can either look at the `sklearn` docs or do this yourself. When you're ready, overwrite the `bank_attributes` column. Make sure that you don't lose the column names in the process.

In [7]:
bank_attributes_scaled = preprocessing.scale(bank_attributes)
bank_attributes = pd.DataFrame(bank_attributes_scaled, columns = bank_attributes.columns)
bank_attributes.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,-1.056270,0.121072,0.374052,-0.711861,-0.576829,-0.407218,-0.320413,-0.343844,-0.514408,-0.196454,...,-0.364805,-0.104676,-0.669064,-0.306828,7.450671,-0.107869,-0.348652,-0.213447,-0.171381,0.469300
1,-0.772583,1.118644,-0.596026,-0.169194,-0.576829,2.989044,2.041734,-0.343844,-0.514408,-0.196454,...,-0.364805,-0.104676,1.494626,-0.306828,-0.134216,-0.107869,2.868193,-0.213447,-0.171381,-2.130831
2,-0.583458,-0.024144,0.010273,-0.303898,-0.576829,2.899143,0.270124,-0.343844,-0.514408,-0.196454,...,-0.364805,-0.104676,-0.669064,-0.306828,-0.134216,-0.107869,2.868193,-0.213447,-0.171381,-2.130831
3,-1.056270,0.017726,-1.566105,-0.250017,0.387967,-0.407218,-0.320413,-0.343844,-0.514408,-0.196454,...,2.741190,-0.104676,-0.669064,-0.306828,-0.134216,-0.107869,-0.348652,-0.213447,-0.171381,0.469300
4,1.686036,-0.472753,-1.323585,-0.146102,-0.576829,-0.407218,-0.320413,-0.343844,1.943983,-0.196454,...,-0.364805,-0.104676,1.494626,-0.306828,-0.134216,-0.107869,-0.348652,-0.213447,-0.171381,0.469300


In [8]:
assert_is_not_none(bank_attributes)

### 3. Split the data (1 point)
Use the standard 70% / 30% split. Since this is a classification problem, be sure to stratify the split according to the `bank_labels`.

In [9]:
bank_attributes_train, bank_attributes_test, bank_labels_train, bank_labels_test = train_test_split(
    bank_attributes, bank_labels, train_size = 0.7, stratify = bank_labels)

In [10]:
assert_is_not_none(bank_attributes_train)
assert_is_not_none(bank_labels_train)

assert_is_not_none(bank_attributes_test)
assert_is_not_none(bank_labels_test)

# TODO: Remove the "Fit it to the train data. Save the trained cross-validator to the variable `k_fold`." text from the bottom cell

### 4. Prepare the cross-validation folds (1 point)
Use a stratified k-fold cross-validation split, with $k = 5$. Fit it to the train data. Save the trained cross-validator to the variable `k_fold`.

The data should already be shuffled. There's no need to shuffle it again.

In [11]:
k_fold = StratifiedKFold(n_splits = 5)

In [12]:
assert_is_not_none(k_fold)

### 5. Decision Tree (2 points)
Use cross-validation to train and optimize the hyperparameters for a decision tree classifier.

Use grid search with the following grid:
* `max_depth`: 1, 5, 7, 15, 20
* `min_samples_leaf`: 2, 5, 10, 12
* `max_leaf_nodes`: 5, 10, 20

Use the most appropriate scoring metric (remember that accuracy doesn't work in this case because the data is highly imbalanced; we need something which combines precision and recall). Use the cross-validation splits you just created.

Save the grid results in `grid_search`. Save the best classifier in `tree_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [13]:
parameters_grid = [{
    "max_depth": [1, 5, 7, 15, 20],
    "min_samples_leaf": [2, 5, 10, 12],
    "max_leaf_nodes": [5, 10, 20]
}]
grid_search = GridSearchCV(DecisionTreeClassifier(), parameters_grid, scoring = "f1", cv = k_fold)
grid_search.fit(bank_attributes_train, bank_labels_train)
tree_classifier = grid_search.best_estimator_

In [14]:
print(grid_search.best_params_)
print("Decision tree; best score:", grid_search.best_score_)

{'max_depth': 7, 'max_leaf_nodes': 10, 'min_samples_leaf': 2}
Decision tree; best score: 0.509000976426


In [15]:
assert_is_not_none(grid_search)
assert_is_not_none(tree_classifier)

### 5. Random Forest (1 point)
Use cross-validation to train and optimize the hyperparameters for a random forest classifier. Use the same technique as before.

Use the following grid:
* `n_estimators`: 100, 200, 300 
* `max_depth`: 20, 50, 100

Note that this grid is on the small side but this is mainly due to performance reasons. Also note that the training will take some time.

Save the grid results in `grid_search`. Save the best classifier in `forest_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

Due to the relatively slow training, we've chosen low values for the parameters. The performance of the random forest will be worse than the decision tree. This is not necessarily the case in general, it's due to the parameters we've chosen to try.

In [16]:
parameters_grid = [{
    "n_estimators": [100, 200, 300],
    "max_depth": [20, 50, 100]
}]
grid_search = GridSearchCV(RandomForestClassifier(), parameters_grid, scoring = "f1", cv = k_fold)
grid_search.fit(bank_attributes_train, bank_labels_train)
forest_classifier = grid_search.best_estimator_

In [17]:
print(grid_search.best_params_)
print("Random forest; best score:", grid_search.best_score_)

{'max_depth': 50, 'n_estimators': 200}
Random forest; best score: 0.392508930899


In [18]:
assert_is_not_none(grid_search)
assert_is_not_none(forest_classifier)

### 6. Linear SVM (1 point)
Use cross-validation to train and optimize the hyperparameters for a linear support vector machine. Use the same technique as before.

Use the following grid:
* `C`: 0.1, 0.5, 0.8, 1, 1.5, 2, 6, 10, 15, 20

Note that we're choosing relatively small values for `C`. This is allowed because our data is normalized.

Save the grid results in `grid_search`. Save the best classifier in `linear_svm_classifier`. There are many ways to create a linear SVM classifier. Look up the `sklearn` docs to choose the fastest one (in terms of performance).

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [19]:
parameters_grid = [{
    "C": [0.1, 0.5, 0.8, 1, 1.5, 2, 6, 10, 15, 20]
}]
grid_search = GridSearchCV(LinearSVC(), parameters_grid, scoring = "f1", cv = k_fold)
grid_search.fit(bank_attributes_train, bank_labels_train)
linear_svm_classifier = grid_search.best_estimator_

In [20]:
print(grid_search.best_params_)
print("Linear SVM; best score:", grid_search.best_score_)

{'C': 20}
Linear SVM; best score: 0.418659438749


In [21]:
assert_is_not_none(grid_search)
assert_is_not_none(linear_svm_classifier)

### 7. Gaussian SVM (1 point)
Use cross-validation to train and optimize the hyperparameters for an SVM with a Gaussian kernel. Use the same technique as before.

Use the following grid:
* `C`: 10, 15, 20, 50, 200
* `gamma`: 0.001, 0.01, 0.1, 0.2

Note that this time we give larger values of `C` because the governing parameter here is `gamma`.

Save the grid results in `grid_search`. Save the best classifier in `gaussian_svm_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [22]:
paramteters_grid = [{
    "C": [10, 15, 20, 50, 200],
    "gamma": [0.001, 0.01, 0.1, 0.2]
}]
grid_search = GridSearchCV(SVC(), paramteters_grid, scoring = "f1", cv = k_fold)
grid_search.fit(bank_attributes_train, bank_labels_train)
gaussian_svm_classifier = grid_search.best_estimator_

In [23]:
print(grid_search.best_params_)
print("Gaussian SVM; best score:", grid_search.best_score_)

{'C': 20, 'gamma': 0.01}
Gaussian SVM; best score: 0.457615843813


In [24]:
assert_is_not_none(grid_search)
assert_is_not_none(gaussian_svm_classifier)

### 9. Compare performance on the testing data (1 point)
Now that you've trained all your models, you've got to select the best one. This should be done on the testing data.

Use the appropriate scoring metric to get the testing scores for all your models. Don't forget to pass the **testing**, not the training data. Save all scores.

Choose the best classifier, based on these scores (the one with the highest test score). Of course, this is not enough. We need to look at ROC curves, track performance through other measures, debug the sources of variance in testing results, try more hyperparameters, etc. However, this is enough for an introductory lab :).

Optionally, you can think of combining them into a boosted model but this is out of the scope of this lab.

In [25]:
tree_classifier_score = f1_score(bank_labels_test, tree_classifier.predict(bank_attributes_test))
forest_classifier_score = f1_score(bank_labels_test, forest_classifier.predict(bank_attributes_test))
linear_svm_classifier_score = f1_score(bank_labels_test, linear_svm_classifier.predict(bank_attributes_test))
gaussian_svm_classifier_score = f1_score(bank_labels_test, gaussian_svm_classifier.predict(bank_attributes_test))

print("Testing scores:")
print("Decision tree:", tree_classifier_score)
print("Random forest:", forest_classifier_score)
print("Linear SVM:", linear_svm_classifier_score)
print("Gaussian SVM:", gaussian_svm_classifier_score)

Testing scores:
Decision tree: 0.462121212121
Random forest: 0.348623853211
Linear SVM: 0.391459074733
Gaussian SVM: 0.438661710037


In [26]:
best_classifier = "tree" # Replace empty string with "tree", "forest", "linear SVM" or "gaussian SVM"

In [27]:
assert_not_equal(best_classifier, "")